# Parameterizing solvent models

## Tutorial setup

Import all the Python modules needed in this tutorial. Most of these are very common in scientific computing, some are popular tools in atomistic simulations.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.io import read
from ase.units import Rydberg, kcal, mol

# Modified version of ASE's write_cube function which also writes atomic charges.
# Original version always assigns zero to atomic charges.
from ase_cube_mod import write_cube

## Reference data

When optimizing solvent model parameters, one of the most common types of reference data are free energies of solvation of small molecules in infinite dilution. This quantity is very straightforward to compute in continuum models, simply as the difference between the energies of the molecule in solution and in vacuum / gas phase.

We have already prepared a small set of reference solvation energies and corresponding molecular geometries, which can be found under ./geometries/

The energies and geometries were taken from the Minnesota Solvation Database (https://comp.chem.umn.edu/mnsol/) and 1.14*CM1A-LBCC atomic charges were obtained using LigParGen (https://zarbi.chem.yale.edu/ligpargen/).

In [ ]:
dG_solv = {
    'ethanol': -5.01,
    'phenol': -6.62,
    'diethylether': -1.76,
    'benzaldehyde': -4.02,
    'acetone': -3.85,
    'aceticacid': -6.7,
    'ethylamine': -4.5,
    'aniline': -5.49,
    'chloroform': -1.07,
    'ethylacetate': -3.1
}

## Setting up the simulations

Let us make a work directory for each solute to keep things clean.

In [ ]:
if not os.path.isdir('work'):
    os.mkdir('work')
for solute in dG_solv.keys():
    dirname = 'work/{}'.format(solute)
    if not os.path.isdir(dirname):
        os.mkdir(dirname)

We need to define a simulation cell around each molecule and write it to a cube file, together with the geometry.

In [ ]:
def get_extent(atoms):
    minval = np.min(atoms.get_positions(), axis=0)
    maxval = np.max(atoms.get_positions(), axis=0)
    return np.max(maxval-minval)

In [ ]:
for solute in dG_solv.keys():
    atoms = read('geometries/{}.xyz'.format(solute))

    # Add at 5 Angstrom to each side
    extent = get_extent(atoms)
    atoms.set_cell((10.+extent) * np.identity(3))
    atoms.set_pbc((True, True, True))
    atoms.center()

    write_cube(open('work/{}/mol.cube'.format(solute), 'w'), atoms, [[[0.]]])

We know that the macroscopic dielectric permittivity of water at 298 K is 78.3, and its surface tension is 72 dyn/cm. Let us test how a solvent model with these values performs. We will ignore pressure for now.

The solvation radius around each atom of the solute depends on its chemical element. Environ comes with different sets of base values for these radii and then scales them by a constant factor alpha. As a starting point, let us use the set that is based on the Universal Force Field (UFF) and set the scaling factor to 1.

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   env_electrostatic = .TRUE.
   env_ecut = 100.
   env_static_permittivity = 78.3
   env_surface_tension = 72.0  ! in dyn/cm
   env_pressure = 0.0   ! in GPa
   !
/
&BOUNDARY
   !
   solvent_mode = 'ionic'
   radius_mode = 'uff'
   alpha = 1.0
   softness = 0.5  ! Spread of the transition region between solvent and cavity
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-2
   !
/
EOF

for dir in work/*/
do
    cp environ.in $dir
done
#clean up
rm environ.in

## Running the simulations

That's all we needed to set up our calculations. Let's run them!

In [ ]:
%%bash

export environ_bin=/work/data/programs_new/Environ/programs/driver

cd work
for dir in *
do
    cd $dir
    echo ''
    echo Running $dir
    orterun -n 4 $environ_bin -n from_cube -c mol.cube --no-density > environ.out
    echo ''
    grep 'total energy' environ.out
    echo ''
    echo '+++++++++++++++++++++++++++++++++++++++++++++++++++++'
    cd ..
done
cd ..

## Evaluating results

Getting a quantity that we can compare to experimental values is very straightforward. The free energy of solvation in infinite dilution is simply the (free) energy difference between the molecule in solution and in vacuum. When running Environ self-consistently with DFT, we would have to repeat our calculations in vacuum. In this tutorial where we use constant atomic charges, it is even simpler. Since the internal energy of the molecule does not change between vacuum and solution, we can simply take the solvent energy contribution from Environ's output.

(NB: If we were really strict, we would still rerun all simulations with the solvent turned off but the pbc correction turned on to correct for its energy contribution. This contributions is, however, relatively small and we will ignore if for this tutorial.)

In [ ]:
dG_exp = []
dG_calc = []
for key, val in dG_solv.items():
    dG_exp.append(val)
    with open('work/{}/environ.out'.format(key), 'r') as ifile:
        for line in reversed(ifile.readlines()):
            if line.startswith('     total energy'):
                
                # Convert to units used in Minnesota Solvation Database
                dG_this = float(line.split()[-2]) * Rydberg / (kcal/mol)
                
                dG_calc.append(dG_this)
                break
dG_exp = np.array(dG_exp)
dG_calc = np.array(dG_calc)

RMSE = np.sqrt(np.mean((dG_exp-dG_calc)**2))
print('RMSE = {:1.2f} kcal/mol'.format(RMSE))

In [ ]:
minplot = min(np.min(dG_exp), np.min(dG_calc)) - 0.2
maxplot = max(np.max(dG_exp), np.max(dG_calc)) + 0.2

plt.plot(dG_exp, dG_calc, 'x')

# Visual guideline for perfect fit
plt.plot([minplot, maxplot], [minplot, maxplot], '--', color='grey')

plt.xlim(minplot, maxplot)
plt.ylim(minplot, maxplot)

plt.xlabel(r'experimental $\Delta G_\mathrm{{solv}}$ in kcal/mol')
plt.ylabel(r'calculated $\Delta G_\mathrm{{solv}}$ in kcal/mol')

plt.gca().set_aspect('equal')

## Improving the model

As you can see, using just the experimental permittivity and surface tension does not predict free energies of solvation very well. The high surface tension of water leads to an overestimation of repulsive interactions and neglects attractive van-der-Waals interactions between solute and solvent.

**Can you find a better parameterization?** Play around with the surface tension, pressure, and radius scaling factor alpha. Hint: The surface tension and pressure in these models are commonly treated as effective parameters with no physical interpretation. As such, they can even be negative.

You have the full power of scipy's optimization tools at your disposal!